In [1]:
print("ok")

ok


In [2]:
%pwd

'/home/motherson/Documents/medical-chatbot/research'

In [4]:
import os
os.chdir('../')

In [5]:
%pwd

'/home/motherson/Documents/medical-chatbot'

In [10]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [11]:
# extract text from pdf files
def load_pdf_files(data):
    loader=DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents=loader.load()
    return documents

In [12]:
extracted_docs=load_pdf_files("data")

In [13]:
extracted_docs[:10]

[Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'data/Medical_book.pdf', 'total_pages': 637, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'data/Medical_book.pdf', 'total_pages': 637, 'page': 1, 'page_label': '2'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION'),
 Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'data/Medical_book.pdf', 'total_pages': 637, 'page': 2, 'page_label': '3'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B\n1'),
 Doc

In [15]:
#filtering operations
from typing import List
from langchain_core.documents import Document


def filter_to_minimal_docs(docs:List[Document]) -> List[Document]:
    """
        Given a list of Document objects,return a new list of Document object containing only 'source' in metadata and the original page_content.
    """
    minimal_docs:List[Document] =[]
    for doc in docs:
        src=doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source":src}
            )
        )
    return minimal_docs

In [16]:
minimal_docs = filter_to_minimal_docs(extracted_docs)

In [17]:
minimal_docs[:10]

[Document(metadata={'source': 'data/Medical_book.pdf'}, page_content=''),
 Document(metadata={'source': 'data/Medical_book.pdf'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION'),
 Document(metadata={'source': 'data/Medical_book.pdf'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B\n1'),
 Document(metadata={'source': 'data/Medical_book.pdf'}, page_content='STAFF\nJacqueline L. Longe, Project Editor\nDeirdre S. Blanchfield, Associate Editor\nChristine B. Jeryan, Managing Editor\nDonna Olendorf, Senior Editor\nStacey Blachford, Associate Editor\nKate Kretschmann, Melissa C. McDade, Ryan\nThomason, Assistant Editors\nMark Springer, Technical Specialist\nAndrea Lopeman, Programmer/Analyst\nBarbara J. Yarrow,Manager, Imaging and Multimedia\nContent\nRobyn V . Young,Project Manager, Imaging and\nMultimedia Content\nDean Dauphinais, Senior Editor, Imaging and\nMultimedia C

In [18]:
#perform chunking operations
#split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk =text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [19]:
texts_chunk=text_split(minimal_docs)
print(f"Number of chunks:{len(texts_chunk)}")

Number of chunks:5860


In [22]:
texts_chunk[5844:]

[Document(metadata={'source': 'data/Medical_book.pdf'}, page_content='tions. Other therapies that deal effectively with muscu-\nloskeletal problems (relating to the muscles and skele-\nton), may also be helpful, such as body work, magnetic\nfield therapy , naturopathic medicine , chiropractic ,\nand applied kinesiology.\nKEY TERMS\nArthritis—Inflammation of a joint that may lead to\nchanges in the joint’s structure. It causes pain and\nswelling. Rheumatoid arthritis is a chronic disease\nthat leads to crippling deformities.\nDiabetes mellitus —A metabolic disease caused'),
 Document(metadata={'source': 'data/Medical_book.pdf'}, page_content='by a deficiency of insulin, which is essential to\nprocess carbohydrates in the body.\nGout—A hereditary metabolic disease that is a\nform of arthritis and causes inflammation of the\njoints. It is more common in men.\nInflammation—The reaction of tissue to injury.\nKinesiology—The science or study of movement.\nPrognosis\nBursitis usually responds

In [26]:
#embedding model
from langchain_huggingface import HuggingFaceEmbeddings


def download_embeddings():
    """
    Download and return the HuggingFace Embeddings model
    """
    model_name="sentence-transformers/all-MiniLM-L6-v2"
    embeddings=HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding=download_embeddings()

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 2160.98it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [27]:
#example to convert data to embeddings 
vector=embedding.embed_query("Hello world")
vector

[-0.03447728604078293,
 0.03102319873869419,
 0.006734993774443865,
 0.02610892988741398,
 -0.039361968636512756,
 -0.16030247509479523,
 0.06692397594451904,
 -0.006441478151828051,
 -0.04745053872466087,
 0.014758829958736897,
 0.07087531685829163,
 0.055527471005916595,
 0.019193343818187714,
 -0.026251329109072685,
 -0.010109460912644863,
 -0.026940539479255676,
 0.022307410836219788,
 -0.022226639091968536,
 -0.1496926099061966,
 -0.017493125051259995,
 0.007676229812204838,
 0.0543522983789444,
 0.00325448764488101,
 0.03172597661614418,
 -0.08462140709161758,
 -0.029405932873487473,
 0.05159565806388855,
 0.048124104738235474,
 -0.0033148033544421196,
 -0.058279216289520264,
 0.04196927696466446,
 0.02221076190471649,
 0.128188818693161,
 -0.022338949143886566,
 -0.011656215414404869,
 0.06292843073606491,
 -0.03287629038095474,
 -0.0912260040640831,
 -0.031175367534160614,
 0.052699532359838486,
 0.047034814953804016,
 -0.08420301973819733,
 -0.030056152492761612,
 -0.020744830

In [28]:
print("vector length dimension:",len(vector))

vector length dimension: 384


In [29]:
#now we will create the knowledge base using pinecone database
from dotenv import load_dotenv
import os
load_dotenv()

True

In [30]:
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY =os.getenv("OPENAI_API_KEY")

os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [31]:
from pinecone import Pinecone 
pinecone_api_key =PINECONE_API_KEY

pc=Pinecone(api_key=pinecone_api_key)

In [32]:
pc

In [33]:
#now create here the index
from pinecone import ServerlessSpec

index_name="medical-chatbot"
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws",region="us-east-1")

    )
index=pc.Index(index_name)


In [34]:
from langchain_pinecone import PineconeVectorStore

docsearch =PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)

In [35]:
#load Existing index 
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [36]:
retriever=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [37]:
retrieved_docs=retriever.invoke("what is acne?")
retrieved_docs

[Document(id='db353585-1fd7-44c4-85a6-eda2814c02ed', metadata={'source': 'data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='82f6f00e-b359-41ef-928f-25b50dd624ad', metadata={'source': 'data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='1d217c66-0280-4409-8dda-9054669ede95', metadata={'source': 'data/Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with 

In [38]:
#this we will pass to LLm 
from langchain_openai import ChatOpenAI

chatmodel=ChatOpenAI(model="gpt-4o")

In [40]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [41]:
system_prompt=(
    "you are a medical assistant for question-answering tasks."
    "use the following pieces of retrieved context to answer "
    "the question. if you don't know the answer ,say that you don't know"
    "use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

In [42]:
system_advance_prompt = """
You are a professional medical knowledge assistant designed for question-answering using retrieved medical documents.

IMPORTANT RULES:

1. Answer ONLY using the provided information.
2. If the answer is not clearly available, respond exactly with:
   "I do not have enough information in the provided documents to answer this question."
3. Do NOT generate or assume medical facts.
4. Do NOT provide a definitive diagnosis.
5. Do NOT prescribe medications or treatment plans.
6. Provide general educational information only.
7. If the question mentions emergency symptoms (e.g., chest pain, difficulty breathing, severe bleeding, loss of consciousness), advise seeking immediate medical attention.
8. Keep responses clear, concise, and medically accurate.
9. Use structured formatting (short paragraphs or bullet points) when helpful.

SAFETY NOTICE:
This assistant is not a replacement for a licensed medical professional. Always encourage consulting a qualified healthcare provider for personalized medical advice.

RESPONSE INSTRUCTIONS:
- Base your answer strictly on the retrieved information.
- If multiple relevant points exist, summarize them clearly.
- Do not mention or reference the source of the information.
- Limit your response to a maximum of three sentences.

{context}
"""


In [43]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [44]:
question_answer_chain=create_stuff_documents_chain(chatmodel,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [45]:
response=rag_chain.invoke({"input":"what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly is a disorder characterized by the abnormal release of a chemical from the pituitary gland in the brain, leading to increased growth in bones and soft tissue and various other disturbances in the body. Gigantism is a similar condition, but it occurs when the abnormality begins before bone growth has stopped, resulting in unusual height. Both conditions are due to an excess of growth hormone but differ based on the timing of hormonal excess relative to bone growth.


In [46]:
advance_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_advance_prompt),
        ("human","{input}")
    ]
)

In [47]:
question_answer_chain=create_stuff_documents_chain(chatmodel,advance_prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [48]:
response=rag_chain.invoke({"input":"what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly is a disorder characterized by the abnormal release of a specific chemical from the pituitary gland in the brain, leading to increased growth in bone and soft tissue, along with various other bodily disturbances. When this abnormality occurs after bone growth has stopped, it is referred to as acromegaly. Gigantism occurs when this abnormality happens before bone growth ceases, resulting in unusual height.


In [49]:
response=rag_chain.invoke({"input":"what is the treatment of acne?"})
print(response["answer"])

The treatment of acne depends on whether it is mild, moderate, or severe. For mild noninflammatory acne, topical treatments such as tretinoin, benzoyl peroxide, adapalene, or salicylic acid can be used to reduce the formation of new comedones. In cases where acne is complicated by inflammation, topical antibiotics might be added. Improvement is usually seen in two to four weeks, but treatment may need to continue for a longer period to maintain results.
